In [7]:
import pandas as pd
from pathlib import Path

# ==================================================
# CONFIG
# ==================================================
RAW_DATA_PATH = Path("..\\data\\raw")
OUTPUT_PATH = Path("D:\\AIProjects\\AI-Powered Patient Health Monitoring & Predictive Analytics System\\data\\processed")
OUTPUT_PATH.mkdir(parents=True, exist_ok=True)
OUTPUT_FILE = OUTPUT_PATH / "final_healthcare_dataset.csv"

# ==================================================
# SCHEMA
# ==================================================
SCHEMA = {
    "patient": {
        "file": "patients.csv",
        "pk": ["patient_id"],
        "columns": {
            "Id": "patient_id",
            "BIRTHDATE": "birthdate",
            "FIRST": "first_name",
            "LAST": "last_name",
            "MARITAL": "marital_status",
            "RACE": "race",
            "ETHNICITY": "ethnicity",
            "GENDER": "gender",
        },
    },
    "encounter": {
        "file": "encounters.csv",
        "pk": ["patient_id", "encounter_id"],
        "columns": {
            "Id": "encounter_id",
            "PATIENT": "patient_id",
            "ENCOUNTERCLASS": "encounter_class",
            "CODE": "encounter_code",
            "DESCRIPTION": "encounter_description",
        },
    },
    "conditions": {
        "file": "conditions.csv",
        "fk": ["patient_id", "encounter_id"],
        "columns": {
            "PATIENT": "patient_id",
            "ENCOUNTER": "encounter_id",
            "CODE": "condition_code",
            "DESCRIPTION": "condition_description",
        },
    },
    "allergies": {
        "file": "allergies.csv",
        "fk": ["patient_id", "encounter_id"],
        "columns": {
            "PATIENT": "patient_id",
            "ENCOUNTER": "encounter_id",
            "CODE": "allergy_code",
            "DESCRIPTION": "allergy_description",
            "TYPE": "allergy_type",
            "CATEGORY": "allergy_category",
        },
    },
    "observations": {
        "file": "observations.csv",
        "fk": ["patient_id", "encounter_id"],
        "columns": {
            "PATIENT": "patient_id",
            "ENCOUNTER": "encounter_id",
            "CATEGORY": "observation_category",
            "CODE": "observation_code",
            "DESCRIPTION": "observation_description",
        },
    },
    "medications": {
        "file": "medications.csv",
        "fk": ["patient_id", "encounter_id"],
        "columns": {
            "PATIENT": "patient_id",
            "ENCOUNTER": "encounter_id",
            "CODE": "medication_code",
            "DESCRIPTION": "medication_description",
        },
    },
    "procedures": {
        "file": "procedures.csv",
        "fk": ["patient_id", "encounter_id"],
        "columns": {
            "PATIENT": "patient_id",
            "ENCOUNTER": "encounter_id",
            "CODE": "procedure_code",
            "DESCRIPTION": "procedure_description",
        },
    },
    "careplans": {
        "file": "careplans.csv",
        "fk": ["patient_id", "encounter_id"],
        "columns": {
            "PATIENT": "patient_id",
            "ENCOUNTER": "encounter_id",
            "CODE": "careplan_code",
            "DESCRIPTION": "careplan_description",
        },
    },
}

# ==================================================
# FUNCTIONS
# ==================================================
def load_dataframe(name, cfg, raw_path=RAW_DATA_PATH):
    """Load CSV, select schema columns, rename, and return DataFrame"""
    path = raw_path / cfg["file"]
    df = pd.read_csv(path, usecols=cfg["columns"].keys())
    df = df.rename(columns=cfg["columns"])
    print(f"{name:<12} loaded {df.shape}")
    return df

def impute_missing(df, strategy=None):
    """Impute missing values in a DataFrame"""
    for col in df.columns:
        if strategy and col in strategy:
            df[col] = df[col].fillna(strategy[col])
        else:
            if pd.api.types.is_numeric_dtype(df[col]):
                df[col] = df[col].fillna(0)
            elif pd.api.types.is_datetime64_any_dtype(df[col]):
                df[col] = df[col].fillna(pd.Timestamp("1900-01-01"))
            else:
                df[col] = df[col].fillna("Unknown")
    return df

def check_dataframe(df, name):
    """Print basic info and sample data for a DataFrame"""
    print(f"\n=== {name.upper()} ===")
    print("Shape:", df.shape)
    print("Columns:", df.columns.tolist())
    print("\nMissing values per column:")
    print(df.isnull().sum())
    print("\nDuplicate rows:", df.duplicated().sum())
    print("\nSample rows:")
    print(df.head(3))
    print("-"*50)

def preprocess_dataframe(name, cfg, strategy=None, check=True):
    """Full preprocessing: load, impute, deduplicate"""
    df = load_dataframe(name, cfg)
    df = impute_missing(df, strategy)
    if "pk" in cfg:
        df = df.drop_duplicates(subset=cfg["pk"])
    if check:
        check_dataframe(df, name)
    return df

def aggregate_child_table(df, keys, agg_columns):
    """Aggregate child tables to have one row per encounter"""
    agg_map = {col: lambda x: ", ".join(sorted(set(x.dropna().astype(str)))) for col in agg_columns}
    return df.groupby(keys, as_index=False).agg(agg_map)

# ==================================================
# LOAD AND PREPROCESS ALL DATAFRAMES
# ==================================================
dfs = {name: preprocess_dataframe(name, cfg) for name, cfg in SCHEMA.items()}

# ==================================================
# AGGREGATE CHILD TABLES
# ==================================================
child_tables = ["conditions", "allergies", "observations", "medications", "procedures", "careplans"]
for table in child_tables:
    cfg = SCHEMA[table]
    agg_columns = list(cfg["columns"].values())
    dfs[table] = aggregate_child_table(dfs[table], keys=cfg["fk"], agg_columns=agg_columns)

# ==================================================
# MERGE CORE + CHILD TABLES
# ==================================================
final_df = dfs["encounter"].merge(dfs["patient"], on="patient_id", how="left")
for table in child_tables:
    final_df = final_df.merge(dfs[table], on=["patient_id","encounter_id"], how="left")

# ==================================================
# VALIDATION
# ==================================================
duplicates = final_df.duplicated(subset=["patient_id","encounter_id"]).sum()
print(f"\nDuplicate encounters: {duplicates}")
assert duplicates == 0, "❌ Duplicate encounters found!"

print("\n✅ Final dataset valid")
print("Final shape:", final_df.shape)

# ==================================================
# SAVE FINAL DATASET
# ==================================================
final_df.to_csv(OUTPUT_FILE, index=False)
print(f"\n📁 Saved to: {OUTPUT_FILE}")




patient      loaded (140, 8)

=== PATIENT ===
Shape: (140, 8)
Columns: ['patient_id', 'birthdate', 'first_name', 'last_name', 'marital_status', 'race', 'ethnicity', 'gender']

Missing values per column:
patient_id        0
birthdate         0
first_name        0
last_name         0
marital_status    0
race              0
ethnicity         0
gender            0
dtype: int64

Duplicate rows: 0

Sample rows:
                             patient_id   birthdate   first_name  \
0  f4db39b1-693d-e301-0a1f-41d26d3ced36  1985-10-13   Charles364   
1  e8503522-82ff-67a2-4dad-29281d6318de  1965-10-07  Leonardo412   
2  ac643af6-1a32-08ce-df40-1c31348bcb41  1965-10-03     Daren950   

       last_name marital_status   race    ethnicity gender  
0    Buckridge80              S  white  nonhispanic      M  
1       Berge125              M  white  nonhispanic      M  
2  Morissette863              M  white  nonhispanic      M  
--------------------------------------------------
encounter    loaded (12

In [8]:
check_dataframe(final_df, "final_healthcare_dataset")


=== FINAL_HEALTHCARE_DATASET ===
Shape: (12361, 27)
Columns: ['encounter_id', 'patient_id', 'encounter_class', 'encounter_code', 'encounter_description', 'birthdate', 'first_name', 'last_name', 'marital_status', 'race', 'ethnicity', 'gender', 'condition_code', 'condition_description', 'allergy_code', 'allergy_description', 'allergy_type', 'allergy_category', 'observation_category', 'observation_code', 'observation_description', 'medication_code', 'medication_description', 'procedure_code', 'procedure_description', 'careplan_code', 'careplan_description']

Missing values per column:
encounter_id                   0
patient_id                     0
encounter_class                0
encounter_code                 0
encounter_description          0
birthdate                      0
first_name                     0
last_name                      0
marital_status                 0
race                           0
ethnicity                      0
gender                         0
condition_code

In [9]:
# Replace NaN with empty string for aggregated child table columns
child_columns = [
    "condition_code", "condition_description",
    "allergy_code", "allergy_description", "allergy_type", "allergy_category",
    "observation_category", "observation_code", "observation_description",
    "medication_code", "medication_description",
    "procedure_code", "procedure_description",
    "careplan_code", "careplan_description"
]

final_df[child_columns] = final_df[child_columns].fillna("")


In [10]:
final_df["has_condition"] = final_df["condition_code"].apply(lambda x: 0 if x=="" else 1)
final_df["has_allergy"] = final_df["allergy_code"].apply(lambda x: 0 if x=="" else 1)
final_df["has_observation"] = final_df["observation_category"].apply(lambda x: 0 if x=="" else 1)


In [11]:
check_dataframe(final_df, "final_healthcare_dataset")


=== FINAL_HEALTHCARE_DATASET ===
Shape: (12361, 30)
Columns: ['encounter_id', 'patient_id', 'encounter_class', 'encounter_code', 'encounter_description', 'birthdate', 'first_name', 'last_name', 'marital_status', 'race', 'ethnicity', 'gender', 'condition_code', 'condition_description', 'allergy_code', 'allergy_description', 'allergy_type', 'allergy_category', 'observation_category', 'observation_code', 'observation_description', 'medication_code', 'medication_description', 'procedure_code', 'procedure_description', 'careplan_code', 'careplan_description', 'has_condition', 'has_allergy', 'has_observation']

Missing values per column:
encounter_id               0
patient_id                 0
encounter_class            0
encounter_code             0
encounter_description      0
birthdate                  0
first_name                 0
last_name                  0
marital_status             0
race                       0
ethnicity                  0
gender                     0
condition_c

In [12]:
final_df

,encounter_id,patient_id,encounter_class,encounter_code,encounter_description,birthdate,first_name,last_name,marital_status,race,...,observation_description,medication_code,medication_description,procedure_code,procedure_description,careplan_code,careplan_description,has_condition,has_allergy,has_observation
0,fa301d15-4761-7c57-6633-5c81cc43d233,f4db39b1-693d-e301-0a1f-41d26d3ced36,wellness,410620009,Well child visit (procedure),1985-10-13,Charles364,Buckridge80,S,white,...,,,,,,,,1,0,0
1,9d4fb2ca-abbd-4270-822a-880cfc94bfb0,f4db39b1-693d-e301-0a1f-41d26d3ced36,inpatient,183452005,Encounter Inpatient,1985-10-13,Charles364,Buckridge80,S,white,...,,,,,,,,1,0,0
2,acde7c73-fba1-be42-58ad-314c7c6465a2,f4db39b1-693d-e301-0a1f-41d26d3ced36,inpatient,183452005,Encounter Inpatient,1985-10-13,Charles364,Buckridge80,S,white,...,,,,,,,,1,0,0
3,aaf60ab9-3f5e-2ff1-8300-a2dbdacb04ce,f4db39b1-693d-e301-0a1f-41d26d3ced36,inpatient,183452005,Encounter Inpatient,1985-10-13,Charles364,Buckridge80,S,white,...,,,,,,,,1,0,0
4,96e42627-d0ab-7e1a-30dc-0731aa49151f,f4db39b1-693d-e301-0a1f-41d26d3ced36,wellness,410620009,Well child visit (procedure),1985-10-13,Charles364,Buckridge80,S,white,...,,,,,,,,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12356,2fb7287e-56f3-bc4a-7f4b-4e9f3f042c76,62cc7665-709d-576e-052a-bdc7eae265be,outpatient,33879002,Administration of vaccine to produce active im...,1928-04-26,Lela622,Bahringer146,M,white,...,,,,,,,,0,0,0
12357,ec82ab7a-4b64-c4d1-13e4-adecec9bd11d,62cc7665-709d-576e-052a-bdc7eae265be,ambulatory,390906007,Follow-up encounter,1928-04-26,Lela622,Bahringer146,M,white,...,Alanine aminotransferase [Enzymatic activity/v...,314231,Simvastatin 10 MG Oral Tablet,,,,,0,0,1
12358,262997ae-a23e-6b89-aefb-76a6c8bdfa85,62cc7665-709d-576e-052a-bdc7eae265be,wellness,162673000,General examination of patient (procedure),1928-04-26,Lela622,Bahringer146,M,white,...,"Are you Hispanic or Latino?, Are you a refugee...","197361, 206905, 309362, 310798, 312961, 314076...","Amlodipine 5 MG Oral Tablet, Clopidogrel 75 MG...","171207006, 428211000124100, 454711000124102, 7...",Assessment of health and social care needs (pr...,,,1,0,1
12359,cdc5fecc-5023-1053-ec60-6ce032edb99b,62cc7665-709d-576e-052a-bdc7eae265be,ambulatory,390906007,Follow-up encounter,1928-04-26,Lela622,Bahringer146,M,white,...,Alanine aminotransferase [Enzymatic activity/v...,314231,Simvastatin 10 MG Oral Tablet,,,,,0,0,1


In [13]:
final_df.to_csv(OUTPUT_FILE, index=False)
print(f"\n📁 Saved to: {OUTPUT_FILE}")


📁 Saved to: D:\AIProjects\AI-Powered Patient Health Monitoring & Predictive Analytics System\data\processed\final_healthcare_dataset.csv
